In [27]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [29]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [30]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [31]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [32]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [33]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [34]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [35]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [36]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [37]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [38]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [39]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [40]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [41]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [42]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [43]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [44]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [45]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [46]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [47]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


In [48]:
# setup
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from scipy.sparse import issparse

In [49]:
RNG = 42
np.random.seed(RNG)

X_tfidf = TfidfTransformer().fit_transform(X_train)
idx_sample = np.random.choice(X_tfidf.shape[0], size=5, replace=False)

def topk_sim_for_doc(i, X_mat, k=5):
    sims = linear_kernel(X_mat[i], X_mat).ravel()
    sims[i] = -1.0
    topk_idx = sims.argsort()[::-1][:k]
    return pd.DataFrame({
        "doc_idx": topk_idx,
        "similarity": sims[topk_idx],
        "label_match": [y_train[i] == y_train[j] for j in topk_idx]
    })

res_list = []
for i in idx_sample:
    df_i = topk_sim_for_doc(i, X_tfidf, k=5)
    df_i.insert(0, "query_idx", i)
    df_i.insert(2, "query_label", y_train[i])
    df_i.insert(4, "doc_label", [y_train[j] for j in df_i["doc_idx"]])
    res_list.append(df_i)

sim_tables_docs = pd.concat(res_list, ignore_index=True)
sim_tables_docs.head(20)


,query_idx,doc_idx,query_label,similarity,doc_label,label_match
0,7492,10935,4,0.783551,4,True
1,7492,7258,4,0.389983,3,False
2,7492,4971,4,0.157331,4,True
3,7492,4303,4,0.128696,6,False
4,7492,8630,4,0.109121,14,False
5,3546,2011,2,0.196060,3,False
6,3546,5665,2,0.190216,3,False
7,3546,8643,2,0.157156,3,False
8,3546,8765,2,0.156673,3,False
9,3546,10706,2,0.142067,3,False


Con TF-IDF y coseno, el vecino más parecido suele “pegarle” bien al tema. Por ejemplo, para la query 7492 (label 4) el top-1 también es label 4 con una similitud muy alta (~0.78). Después, en el top-5 aparecen textos de clases cercanas (labels 3, 6, 14) pero con similitudes mucho más bajas, algo normal cuando comparten vocabulario general. Un caso que llama la atención es la query 3546 (label 2), donde los cinco vecinos caen en la clase 3: eso sugiere solapamiento fuerte entre esas dos clases o mucha variación interna.

In [50]:
#punto 2
idx = np.arange(X_train.shape[0])
X_tr_idx, X_te_idx, y_tr, y_te = train_test_split(idx, y_train, test_size=0.25, random_state=RNG, stratify=y_train)
X_tr_raw = X_train[X_tr_idx]
X_te_raw = X_train[X_te_idx]
tf = TfidfTransformer()
Xtr = tf.fit_transform(X_tr_raw)
Xte = tf.transform(X_te_raw)

sims = linear_kernel(Xte, Xtr)
nn_idx = sims.argmax(axis=1)
y_pred_nn = np.array(y_tr)[nn_idx]

print(classification_report(y_te, y_pred_nn, digits=3))
print("f1_macro:", f1_score(y_te, y_pred_nn, average="macro"))

              precision    recall  f1-score   support

           0      0.624     0.650     0.637       120
           1      0.634     0.568     0.599       146
           2      0.601     0.581     0.591       148
           3      0.507     0.507     0.507       148
           4      0.551     0.604     0.576       144
           5      0.662     0.676     0.669       148
           6      0.540     0.466     0.500       146
           7      0.549     0.564     0.556       149
           8      0.645     0.607     0.625       150
           9      0.651     0.738     0.692       149
          10      0.723     0.713     0.718       150
          11      0.739     0.705     0.722       149
          12      0.634     0.574     0.603       148
          13      0.820     0.705     0.758       149
          14      0.436     0.649     0.522       148
          15      0.643     0.673     0.658       150
          16      0.696     0.640     0.667       136
          17      0.738    

El método para asignar la clase del documento mas parecido dio un f1_macro ~0.62. Para un baseline está bien: funciona cuando hay documentos muy representativos por clase, pero se cae si las clases estan muy cerca semánticamente o son muy diversas por dentro. El reporte muestra esa desigualdad entre clases (algunas rinden mejor que otras), lo que encaja con la idea de solapamiento y de que un único vecino puede no ser un buen prototipo”en todos los casos.

In [51]:
#punto 3
idx = np.arange(X_train.shape[0])
X_tr_idx, X_te_idx, y_tr, y_te = train_test_split(idx, y_train, test_size=0.25, random_state=RNG, stratify=y_train)
X_tr_raw = X_train[X_tr_idx]
X_te_raw = X_train[X_te_idx]
pipe_mnb = Pipeline([("tf", TfidfTransformer()), ("clf", MultinomialNB())])
pipe_cnb = Pipeline([("tf", TfidfTransformer()), ("clf", ComplementNB())])
grid_common = {
    "tf__use_idf": [True, False],
    "tf__smooth_idf": [True, False],
    "tf__sublinear_tf": [False, True],
    "tf__norm": ["l2", None],
}

param_mnb = dict(grid_common, **{"clf__alpha": [0.1, 0.5, 1.0], "clf__fit_prior": [True, False]})
param_cnb = dict(grid_common, **{"clf__alpha": [0.1, 0.5, 1.0], "clf__fit_prior": [True, False]})

gs_mnb = GridSearchCV(pipe_mnb, param_mnb, scoring="f1_macro", cv=5, n_jobs=-1, verbose=0)
gs_cnb = GridSearchCV(pipe_cnb, param_cnb, scoring="f1_macro", cv=5, n_jobs=-1, verbose=0)

gs_mnb.fit(X_tr_raw, y_tr)
gs_cnb.fit(X_tr_raw, y_tr)

pred_mnb = gs_mnb.best_estimator_.predict(X_te_raw)
pred_cnb = gs_cnb.best_estimator_.predict(X_te_raw)

from sklearn.metrics import classification_report, f1_score
score_mnb = f1_score(y_te, pred_mnb, average="macro")
score_cnb = f1_score(y_te, pred_cnb, average="macro")

print("best mnb f1_macro:", round(score_mnb, 4))
print("best cnb f1_macro:", round(score_cnb, 4))
print("\nMNB report:\n", classification_report(y_te, pred_mnb, digits=3))
print("\nCNB report:\n", classification_report(y_te, pred_cnb, digits=3))
print("winner:", "MNB" if score_mnb >= score_cnb else "CNB")

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, *

best mnb f1_macro: 0.756
best cnb f1_macro: 0.7593

MNB report:
               precision    recall  f1-score   support

           0      0.450     0.675     0.540       120
           1      0.726     0.726     0.726       146
           2      0.815     0.655     0.727       148
           3      0.675     0.743     0.707       148
           4      0.762     0.757     0.760       144
           5      0.790     0.838     0.813       148
           6      0.785     0.726     0.754       146
           7      0.852     0.732     0.787       149
           8      0.870     0.800     0.833       150
           9      0.875     0.893     0.884       149
          10      0.924     0.887     0.905       150
          11      0.814     0.792     0.803       149
          12      0.806     0.703     0.751       148
          13      0.913     0.846     0.878       149
          14      0.770     0.791     0.780       148
          15      0.652     0.900     0.756       150
          16    

En este caso se nota la mejora donde MNB ronda f1_macro ~0.756 y CNB ~0.759, con una ventaja chica pero consistente para ComplementNB. Tiene sentido ya que en texto multiclase, CNB suele manejar mejor el desbalance y descontar términos comunes que no ayudan a diferenciar. Los avisos de combinaciones inválidas vienen de algunos parámetros del grid que no van con NB; el search los descarta y no afectan al mejor modelo.

In [54]:
TERM_SEEDS = ["inflation", "prices", "market", "policy", "growth"]

def _find_vectorizer_in_globals():
    cand = []
    for name, obj in globals().items():
        if hasattr(obj, "get_feature_names_out"):
            try:
                _ = obj.get_feature_names_out()
                cand.append(obj)
            except Exception:
                pass
    return cand[0] if cand else None

def term_similarity(X_source, term_seeds=None, topk=5):
    if issparse(X_source):
        X_counts_or_tfidf = X_source
        tf = TfidfTransformer()
        X_tfidf = tf.fit_transform(X_counts_or_tfidf)
        TD = X_tfidf.T
        vec = _find_vectorizer_in_globals()
        if vec is not None:
            try:
                names = np.array(vec.get_feature_names_out())
            except Exception:
                names = np.array([f"f{i}" for i in range(TD.shape[0])])
        else:
            names = np.array([f"f{i}" for i in range(TD.shape[0])])
        col_freq = np.asarray(X_counts_or_tfidf.sum(axis=0)).ravel()
        default_seeds = names[np.argsort(col_freq)[::-1][:5]].tolist()
    else:
        cnt = CountVectorizer()
        X_cnt = cnt.fit_transform(X_source)
        names = np.array(cnt.get_feature_names_out())
        col_freq = np.asarray(X_cnt.sum(axis=0)).ravel()
        tfidf = TfidfVectorizer()
        X_tfidf = tfidf.fit_transform(X_source)
        TD = X_tfidf.T
        default_seeds = names[np.argsort(col_freq)[::-1][:5]].tolist()

    seeds = term_seeds if term_seeds else default_seeds

    name_to_idx = {n:i for i,n in enumerate(names)}
    out = []
    for s in seeds:
        if isinstance(s, int):
            i = s if 0 <= s < TD.shape[0] else None
        else:
            i = name_to_idx.get(s, None)
        if i is None:
            continue
        sims = cosine_similarity(TD[i], TD).ravel()
        sims[i] = -1.0
        idx = sims.argsort()[::-1][:topk]
        terms = names[idx].tolist()
        out.append(pd.DataFrame({"query_term": names[i], "term": terms, "similarity": sims[idx]}))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

sim_tables_terms = term_similarity(X_train, term_seeds=None, topk=5)
sim_tables_terms.head(20)

sim_tables_terms = term_similarity(X_train, term_seeds=TERM_SEEDS, topk=5)
sim_tables_terms.head(20)

,query_term,term,similarity
0,inflation,fv,0.799885
1,inflation,measly,0.526053
2,inflation,speculators,0.480622
3,inflation,salary,0.389040
4,inflation,mortgage,0.355063
5,prices,edmund,0.294779
6,prices,thatn,0.238674
7,prices,authorised,0.231423
8,prices,guides,0.222430
9,prices,mailorder,0.205158


Con las semillas inflation, prices, market, policy los vecindarios mejoraron respecto de la corrida automática previa (que devolvía solo stopwords). Se ven términos que sí tienen relación temática, por ejemplo con inflation aparecen salary y mortgage, y con policy surgen adjetivos y calificativos como unfavorable o laudable que suelen co-ocurrir en textos de opinión. Al mismo tiempo, aparecen ruidos y rarezas del vocabulario (fv, thatn, stragegy, resoltuion, belike, nombres propios como edmund, o irish con market). Eso sugiere que el corpus incluye typos, tokens poco frecuentes y entidades propias que arrastran similitud por co-ocurrencia, aun si no aportan significado directo. En síntesis, al elegir palabras de contenido mejora la interpretabilidad de los top-5 similares, pero la calidad final todavía depende del ruido del vocabulario disponible: conviven vecinos útiles (ej., salary, mortgage con inflation) con términos residuales o poco interpretables que reflejan la dispersión y errores ortográficos presentes en los textos.